In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.41.3 transformers==4.41.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 114.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 90.2 MB/s eta 0:00:00:00:01


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# 4-bit quantization settings
use_4bit = True
bnb_4bit_compute_dtype = torch.float16
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
device_map = "auto"  # Automatically map to available GPUs

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=use_nested_quant,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load base model with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    load_in_4bit=use_4bit,  # Corrected parameter
    low_cpu_mem_usage=True,
    return_dict=True,
    device_map=device_map,  # Fixed device map
    trust_remote_code=True,
    torch_dtype=bnb_4bit_compute_dtype,
)

# Load LoRA adapter
adapter = "Soham7021/text-sql-llm_Lama"
model = PeftModel.from_pretrained(base_model, adapter)

# Ensure model is in evaluation mode
model.eval()

print("Model and LoRA adapter loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

Model and LoRA adapter loaded successfully!


/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [10]:


input_text = "tell me the number of times he placed 4th."
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to("cuda")

with torch.no_grad():
    output = model.generate(**inputs, max_length=500)

# Decode the output tokens to text
generated_sql = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated SQL Query:", generated_sql)

# Safe extraction (agar needed ho)
if '[/INST]' in generated_sql:
    extracted_sql = generated_sql.split('[/INST]')[1].split(';')[0] + ';'
else:
    extracted_sql = generated_sql

print("Extracted SQL:", extracted_sql)


Generated SQL Query: tell me the number of times he placed 4th. Context: CREATE TABLE table_13776 (
    "Year" real,
    "Race" text,
    "Distance" text,
    "Track" text,
    "Winner" text,
    "Time" text,
    "Pole Position" text,
    "Fastest Lap" text,
    "Points" real,
    "1st Place" text,
    "1st Pos." text,
    "Points for 1st" text,
    "Points for 2nd" text,
    "Points for 3rd" text,
    "Points for 4th" text,
    "Points for 5th" text,
    "Points for 6th" text,
    "Points for 7th" text,
    "Points for 8th" text,
    "Points for 9th" text,
    "Points for 10th" text
) [/] SELECT COUNT(*) FROM table_13776 WHERE "4th" = '4th' AND "Points for 4th" = '4th' AND "Pole Position" = 'Pole Position' AND "Winner" = 'Winner' AND "Time" = 'Time' AND "Distance" = 'Distance' AND "Track" = 'Track' AND "1st Place" = '1st Place' AND "Fastest Lap" = 'Fastest Lap' AND "Points" > '15' AND "Pole Position" = 'Pole Position' AND "1st Place" = '1st Place' AND "Fastest Lap" = 'Fastest Lap' AND

In [11]:
import torch
import re

def clean_generated_sql(generated_sql):
    """
    Cleans the generated SQL output to extract the relevant query, handling repetitions.
    """
    cleaned_sql = generated_sql.replace("[/INST]", "").replace("[/access_token]", "").replace("[/]", "")
    select_index = cleaned_sql.find("SELECT")

    if select_index != -1:
        extracted_sql = cleaned_sql[select_index:].strip()
        if ";" in extracted_sql:
            extracted_sql = extracted_sql.split(";")[0].strip() + ";"

        # Remove repetitions within the SQL query itself using regular expressions:
        sql_parts = re.split(r'AND\s+(?=[A-Z"]+)', extracted_sql) #splits on AND followed by a capital letter or quote.
        unique_parts = []
        seen_parts = set()
        for part in sql_parts:
            if part.strip() not in seen_parts:
                unique_parts.append(part.strip())
                seen_parts.add(part.strip())
        extracted_sql = " AND ".join(unique_parts)

        # Remove potential repetitions of the entire SELECT statement:
        parts = extracted_sql.split("SELECT")
        if len(parts) > 1:
            extracted_sql = "SELECT" + parts[1] # only take the first select statement after the first instance of select
        return extracted_sql
    else:
        return cleaned_sql.strip()

# Assuming you have 'tokenizer', 'model', and 'input_text' defined:
input_text = "tell me the number of times he placed 4th."
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to("cuda")

with torch.no_grad():
    output = model.generate(**inputs, max_length=500)

generated_sql = tokenizer.decode(output[0], skip_special_tokens=True)

final_sql = clean_generated_sql(generated_sql)
print("Cleaned SQL Query:", final_sql)

Cleaned SQL Query: SELECT SUM(times_4th) FROM table_10003662_1 WHERE times_4th = 4 AS INTEGER END;


In [12]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [13]:
from datasets import load_dataset
import sqlite3
import re
from difflib import SequenceMatcher

# Dataset load kar rahe hain
dataset = load_dataset("Clinton/texttosqlv2_25000_v2", split="train")

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

texttosqlv2_25000_v2.csv:   0%|          | 0.00/56.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [14]:
def extract_sql_query(output_text):
    """
    Model output se SQL query extract karta hai
    Format mein agar '[/]' ke baad SQL hai, toh wahi part nikaalta hai.
    """
    match = re.search(r'\[/\] (.*)', output_text, re.DOTALL)
    return match.group(1).strip() if match else output_text.strip()

In [16]:
def similarity(a, b):
    """
    Do strings ke beech similarity ratio nikaalta hai (0 se 1 ke beech)
    """
    return SequenceMatcher(None, a, b).ratio()

In [17]:
def evaluate_and_get_top(model, tokenizer, total_records=1000, top_n=100, use_db=False, db_path=None):
    """
    Model ko test karta hai aur top N records deta hai jinke highest similarity scores hote hain.
    """
    test_samples = dataset.select(range(total_records))
    results = []

    # DB setup agar zarurat ho
    conn = None
    if use_db and db_path:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

    for sample in test_samples:
        input_text = sample["input"]
        true_sql = sample["output"].strip()

        # Tokenize and prediction
        inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
        # outputs = model.generate(**inputs, max_length=500)
        outputs = model.generate(**inputs, max_new_tokens=100)
        predicted_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Clean prediction
        predicted_sql = extract_sql_query(predicted_output)

        # Similarity calculation
        sim_score = similarity(predicted_sql.lower().strip(), true_sql.lower().strip())

        # Results append
        results.append({
            "input": input_text,
            "predicted_sql": predicted_sql,
            "true_sql": true_sql,
            "similarity": sim_score
        })

    if conn:
        conn.close()

    # Sort by similarity score
    sorted_results = sorted(results, key=lambda x: x["similarity"], reverse=True)
    top_results = sorted_results[:top_n]

    # Print Top 100
    print(f"Top {top_n} best matched records:\n" + "=" * 60)
    for idx, record in enumerate(top_results, start=1):
        print(f"{idx}. Input: {record['input']}")
        print(f"   Predicted SQL: {record['predicted_sql']}")
        print(f"   True SQL: {record['true_sql']}")
        print(f"   Similarity Score: {record['similarity']}")
        print("-" * 60)

    return top_results


In [18]:
top_100_records = evaluate_and_get_top(model, tokenizer, total_records=1000, top_n=100, use_db=True, db_path="your_database.db")


Top 100 best matched records:
1. Input: CREATE TABLE table_name_32 (
    name VARCHAR,
    year VARCHAR
)
   Predicted SQL: SELECT name FROM table_name_32 WHERE year = 2012;
   True SQL: SELECT name FROM table_name_32 WHERE year = "2001"
   Similarity Score: 0.9494949494949495
------------------------------------------------------------
2. Input: CREATE TABLE table_1012730_1 (
    avg_finish VARCHAR,
    year VARCHAR
)
   Predicted SQL: SELECT avg_finish FROM table_1012730_1 WHERE year = 1990;
   True SQL: SELECT avg_finish FROM table_1012730_1 WHERE year = 2012
   Similarity Score: 0.9380530973451328
------------------------------------------------------------
3. Input: CREATE TABLE table_name_7 (
    position VARCHAR,
    pick VARCHAR
)
   Predicted SQL: SELECT position FROM table_name_7 WHERE pick = 7;
   True SQL: SELECT position FROM table_name_7 WHERE pick = "146"
   Similarity Score: 0.9306930693069307
------------------------------------------------------------
4. Input: CREATE

In [19]:
import json
import re

# Tumhare data ko yahan paste karo
data = '''
   Predicted SQL: SELECT rate_limit__p_ FROM table_25316812_1 WHERE desired_rate_change___percentage_ = "10"
   True SQL: SELECT rate_limit__p_ FROM table_25316812_1 WHERE desired_rate_change___percentage_ = "+40.4"
   Similarity Score: 0.9726775956284153

   Predicted SQL: SELECT position FROM table_26996293_2 WHERE cfl_team = "Edmonton"
   True SQL: SELECT position FROM table_26996293_2 WHERE cfl_team = "Edmonton (2)"
   Similarity Score: 0.9701492537313433

   Predicted SQL: SELECT name FROM table_name_32 WHERE year = 2010;
   True SQL: SELECT name FROM table_name_32 WHERE year = "2001"
   Similarity Score: 0.9494949494949495

   Predicted SQL: SELECT position FROM table_name_7 WHERE pick = 7;
   True SQL: SELECT position FROM table_name_7 WHERE pick = "146"
   Similarity Score: 0.9306930693069307

   Predicted SQL: SELECT voltage_center__v_ FROM table_name_76 WHERE s_spec_number = "110/220v"
   True SQL: SELECT voltage_center__v_ FROM table_name_76 WHERE s_spec_number = "sk096"
   Similarity Score: 0.9271523178807947

   Predicted SQL: SELECT theme FROM table_name_59 WHERE week = 1;
   True SQL: SELECT theme FROM table_name_59 WHERE week = "top 11"
   Similarity Score: 0.92

   Predicted SQL: SELECT partnered_with FROM table_name_97 WHERE score_in_final = "2–6, 6–4, 7–6(7–4)";
   True SQL: SELECT partnered_with FROM table_name_97 WHERE score_in_final = "4–6, 3–6"
   Similarity Score: 0.9056603773584906

   Predicted SQL: SELECT candidates FROM table_1341973_3 WHERE district = 3;
   True SQL: SELECT candidates FROM table_1341973_3 WHERE district = "Alabama 6"
   Similarity Score: 0.896

   Predicted SQL: SELECT race_2 FROM table_name_6 WHERE race_1 = "3rd" LIMIT 1;
   True SQL: SELECT race_2 FROM table_name_6 WHERE race_1 = "30"
   Similarity Score: 0.8928571428571429

   Predicted SQL: SELECT bullet_weight FROM table_16010376_1 WHERE source = "Creighton University"
   True SQL: SELECT bullet_weight FROM table_16010376_1 WHERE source = "Weatherby"
   Similarity Score: 0.87248322147651

   Predicted SQL: SELECT team_captain FROM table_23214833_1 WHERE capacity = 1200000 OFFSET 0 END;
   True SQL: SELECT team_captain FROM table_23214833_1 WHERE capacity = 17800
   Similarity Score: 0.8611111111111112

   Predicted SQL: SELECT driver_s_ FROM table_name_57 WHERE owner_s_ = "bob kennedy" LIMIT 1;
   True SQL: SELECT driver_s_ FROM table_name_57 WHERE owner_s_ = "bob keselowski"
   Similarity Score: 0.8611111111111112

   Predicted SQL: SELECT date_of_vacancy FROM table_17115950_2 WHERE replaced_by = "Sir John F. Kennedy"
   True SQL: SELECT date_of_vacancy FROM table_17115950_2 WHERE replaced_by = "Viorel Moldovan"
   Similarity Score: 0.8571428571428571

   Predicted SQL: SELECT us_exclusive FROM table_24600706_1 WHERE artist_band = "Kings of Leon" END;
   True SQL: SELECT us_exclusive FROM table_24600706_1 WHERE artist_band = "Miley Cyrus"
   Similarity Score: 0.8535031847133758

   Predicted SQL: SELECT week_4 FROM table_name_62 WHERE week_2 = "tie" END;
   True SQL: SELECT week_4 FROM table_name_62 WHERE week_2 = "samantha speer"
   Similarity Score: 0.8524590163934426

   Predicted SQL: SELECT attendance FROM table_name_34 WHERE date = "week 10" AND week = 10;
   True SQL: SELECT COUNT(attendance) FROM table_name_34 WHERE date = "december 13, 1970" AND week > 13
   Similarity Score: 0.8414634146341463

   Predicted SQL: SELECT mixed_doubles FROM table_15002265_1 WHERE mens_singles = "Men's singles" END;
   True SQL: SELECT mixed_doubles FROM table_15002265_1 WHERE mens_singles = "Peter Moritz"
   Similarity Score: 0.8395061728395061

   Predicted SQL: SELECT notes FROM table_name_24 WHERE athlete = "aleksandr abt" LIMIT 1;
   True SQL: SELECT notes FROM table_name_24 WHERE athlete = "andre vonarburg"
   Similarity Score: 0.8321167883211679

   Predicted SQL: SELECT name FROM table_name_56 WHERE number_of_electorates__2009_ = 56 ORDER BY name LIMIT 1 OFFSET 0
   True SQL: SELECT name FROM table_name_56 WHERE number_of_electorates__2009_ > 188 OFFSET 799
   Similarity Score: 0.8306010928961749

   Predicted SQL: SELECT venue FROM table_name_19 WHERE date = "2013-05-05" END;
   True SQL: SELECT venue FROM table_name_19 WHERE date = "24/11/1979"
   Similarity Score: 0.8235294117647058

   Predicted SQL: SELECT country FROM table_name_75 WHERE player = "andy roddick" LIMIT 1;
   True SQL: SELECT country FROM table_name_75 WHERE player = "jack nicklaus"
   Similarity Score: 0.8235294117647058

   Predicted SQL: SELECT position FROM table_name_85 WHERE league = "nhl" AND year = 1974;
   True SQL: SELECT position FROM table_name_85 WHERE league = "malaysian super league" AND year = "2011"
   Similarity Score: 0.8170731707317073

   Predicted SQL: SELECT laps FROM table_name_38 WHERE grid = 18 OFFSET 18 END;
   True SQL: SELECT laps FROM table_name_38 WHERE grid < 2
   Similarity Score: 0.8113207547169812

   Predicted SQL: SELECT MIN(capacity) FROM table_25794138_1 WHERE stadium = "Olympiastadion" LIMIT 1;
   True SQL: SELECT MAX(capacity) FROM table_25794138_1 WHERE stadium = "Larry Gomes stadium"
   Similarity Score: 0.8048780487804879

   Predicted SQL: SELECT date FROM table_name_18 WHERE works_number = "241" ORDER BY date LIMIT 1;
   True SQL: SELECT date FROM table_name_18 WHERE works_number = "7"
   Similarity Score: 0.8

   Predicted SQL: SELECT AVG(avg) FROM table_name_70 WHERE long = "8000" INTEGER;
   True SQL: SELECT SUM(avg) FROM table_name_70 WHERE long = 32
   Similarity Score: 0.7964601769911505

   Predicted SQL: SELECT icao FROM table_name_66 WHERE airport = "beijing" LIMIT 1;
   True SQL: SELECT icao FROM table_name_66 WHERE airport = "luqa airport"
   Similarity Score: 0.7936507936507936

   Predicted SQL: SELECT player FROM table_name_89 WHERE to_par = "–" AND score = "–" ORDER BY player LIMIT 1;
   True SQL: SELECT player FROM table_name_89 WHERE to_par = "+1" AND score = 71 - 67 - 73 = 211
   Similarity Score: 0.7885714285714286

   Predicted SQL: SELECT location FROM table_name_46 WHERE unit = "create" LIMIT 1;
   True SQL: SELECT location FROM table_name_46 WHERE unit = "two medicine formation"
   Similarity Score: 0.7883211678832117

   Predicted SQL: SELECT player FROM table_name_43 WHERE school = "central" LIMIT 1;
   True SQL: SELECT player FROM table_name_43 WHERE school = "vanderbilt university"
   Similarity Score: 0.7883211678832117

   Predicted SQL: SELECT season FROM table_name_35 WHERE rank = "1" ORDER BY season LIMIT 1;
   True SQL: SELECT season FROM table_name_35 WHERE rank = "2"
   Similarity Score: 0.7804878048780488

   Predicted SQL: SELECT conf FROM table_name_25 WHERE original_nfl_team = "chicago bears" ORDER BY conf LIMIT 1;
   True SQL: SELECT conf FROM table_name_25 WHERE original_nfl_team = "denver broncos"
   Similarity Score: 0.7738095238095238

   Predicted SQL: SELECT stadium FROM table_name_35 WHERE final_score = "35" ORDER BY stadium LIMIT 1;
   True SQL: SELECT stadium FROM table_name_35 WHERE final_score = "20-10"
   Similarity Score: 0.7724137931034483

   Predicted SQL: SELECT rank_in_spanish FROM table_1015521_2 WHERE rank_in_english = "First" ORDER BY rank_in_spanish LIMIT 1;
   True SQL: SELECT rank_in_spanish FROM table_1015521_2 WHERE rank_in_english = "Major"
   Similarity Score: 0.7717391304347826

   Predicted SQL: SELECT season FROM table_name_6 WHERE rating = "18.2" ORDER BY season LIMIT 1;
   True SQL: SELECT season FROM table_name_6 WHERE rating = "+98"
   Similarity Score: 0.7692307692307693

   Predicted SQL: SELECT team_2 FROM table_name_49 WHERE team_2 = "maccabi tel aviv"
   True SQL: SELECT 2 AS nd_leg FROM table_name_49 WHERE team_2 = "barcelona"
   Similarity Score: 0.7692307692307693

   Predicted SQL: SELECT away_team FROM table_name_12 WHERE home_team = "chelsea" AND competition = "premier league" ORDER BY away_team LIMIT 1;
   True SQL: SELECT away_team FROM table_name_12 WHERE home_team = "leeds united" AND competition = "league cup"
   Similarity Score: 0.7644444444444445

   Predicted SQL: SELECT * FROM table_name_96 WHERE election = "2012" LIMIT 1;
   True SQL: SELECT 2 AS nd_party FROM table_name_96 WHERE election = "1837"
   Similarity Score: 0.7642276422764228

   Predicted SQL: SELECT tie_no FROM table_name_46 WHERE home_team = "melbourne" ORDER BY tie_no LIMIT 1;
   True SQL: SELECT tie_no FROM table_name_46 WHERE home_team = "middlesbrough"
   Similarity Score: 0.7581699346405228

   Predicted SQL: SELECT T1.Code FROM Products AS T1 JOIN Manufacturers AS T2 ON T1.Manufacturer = T2.Code WHERE T2.Name = 'Sony'
   True SQL: SELECT Founder, SUM(Revenue) FROM Products AS T1 JOIN Manufacturers AS T2 ON T1.Manufacturer = T2.Code GROUP BY Founder
   Similarity Score: 0.7565217391304347

   Predicted SQL: SELECT rowers FROM table_name_80 WHERE country = "uk" ORDER BY rowers LIMIT 1;
   True SQL: SELECT rowers FROM table_name_80 WHERE country = "canada"
   Similarity Score: 0.7555555555555555

   Predicted SQL: SELECT total FROM table_name_33 WHERE player = "jim furyk" ORDER BY total LIMIT 1;
   True SQL: SELECT total FROM table_name_33 WHERE player = "tony jacklin"
   Similarity Score: 0.7552447552447552

   Predicted SQL: SELECT away_captain FROM table_name_83 WHERE result = "win" AND date = "2010-11-07" ORDER BY away_captain LIMIT 1
   True SQL: SELECT away_captain FROM table_name_83 WHERE result = "draw" AND date = "12,13,14 jun 1902"
   Similarity Score: 0.7549019607843137

   Predicted SQL: SELECT tie_no FROM table_name_53 WHERE away_team = "chelsea" ORDER BY tie_no LIMIT 1;
   True SQL: SELECT tie_no FROM table_name_53 WHERE away_team = "newcastle united"
   Similarity Score: 0.7532467532467533

   Predicted SQL: SELECT original_airdate FROM table_2501754_2 WHERE prod_code = "CREATE" ORDER BY original_airdate LIMIT 1
   True SQL: SELECT original_airdate FROM table_2501754_2 WHERE prod_code = "IPEA345A"
   Similarity Score: 0.7528089887640449

   Predicted SQL: SELECT head_coach FROM table_14015965_1 WHERE previous_season = "2018" GROUP BY head_coach HAVING COUNT(*) = 1;
   True SQL: SELECT head_coach FROM table_14015965_1 WHERE previous_season = "2"
   Similarity Score: 0.7528089887640449

   Predicted SQL: SELECT frequency FROM table_1601792_3 WHERE station = "KKCW" ORDER BY frequency LIMIT 1;
   True SQL: SELECT COUNT(frequency) FROM table_1601792_3 WHERE station = "Hot FM"
   Similarity Score: 0.7515923566878981

   Predicted SQL: SELECT district FROM table_1341884_19 WHERE incumbent = "John Lewis" AND district = "5" LIMIT 1;
   True SQL: SELECT district FROM table_1341884_19 WHERE incumbent = "Frank Chelf"
   Similarity Score: 0.7515151515151515

   Predicted SQL: SELECT founded FROM table_262505_1 WHERE current_conference = "Big Ten" ORDER BY founded LIMIT 10 OFFSET 0;
   True SQL: SELECT founded FROM table_262505_1 WHERE current_conference = "school closed in 2005"
   Similarity Score: 0.75

   Predicted SQL: SELECT växjö FROM table_name_53 WHERE karlstad = "karlstad" AND växjö = "växjö" END;
   True SQL: SELECT växjö FROM table_name_53 WHERE karlstad = "1"
   Similarity Score: 0.75

   Predicted SQL: SELECT purse FROM table_name_21 WHERE trainer = "michael stidham" ORDER BY purse LIMIT 1;
   True SQL: SELECT purse FROM table_name_21 WHERE trainer = "aidan o'brien"
   Similarity Score: 0.75

   Predicted SQL: SELECT team FROM table_name_74 WHERE pick = 74 AND college = "ohio state" ORDER BY pick LIMIT 1;
   True SQL: SELECT team FROM table_name_74 WHERE pick > 30 AND college = "saginaw valley state"
   Similarity Score: 0.7486033519553073

   Predicted SQL: SELECT venue FROM table_name_96 WHERE result = "create" ORDER BY venue LIMIT 1;
   True SQL: SELECT venue FROM table_name_96 WHERE result = "1–0"
   Similarity Score: 0.7480916030534351

   Predicted SQL: SELECT capacity FROM table_name_97 WHERE stadium = "stade de france" LIMIT 1;
   True SQL: SELECT COUNT(capacity) FROM table_name_97 WHERE stadium = "alberto picco"
   Similarity Score: 0.7466666666666667

   Predicted SQL: SELECT position FROM table_25017530_6 WHERE player = "Nick Young" ORDER BY position LIMIT 1;
   True SQL: SELECT position FROM table_25017530_6 WHERE player = "Michael Warner"
   Similarity Score: 0.7453416149068323

   Predicted SQL: SELECT incumbent FROM table_name_93 WHERE party = "republican" AND district = "93" ORDER BY incumbent LIMIT 1;
   True SQL: SELECT incumbent FROM table_name_93 WHERE party = "democratic" AND district = "ohio 7"
   Similarity Score: 0.7448979591836735

   Predicted SQL: SELECT country FROM table_name_49 WHERE score = "8.5" ORDER BY country LIMIT 1;
   True SQL: SELECT country FROM table_name_49 WHERE score = 79 - 68 - 74 = 212
   Similarity Score: 0.7448275862068966

   Predicted SQL: SELECT college FROM table_11677691_8 WHERE player = "Jeremy Smith" ORDER BY college LIMIT 1;
   True SQL: SELECT college FROM table_11677691_8 WHERE player = "Robert Nkemdiche ‡"
   Similarity Score: 0.7439024390243902

   Predicted SQL: SELECT runner_s__up FROM table_name_67 WHERE margin_of_victory = "2" AND date = "august 23" ORDER BY runner_s__up LIMIT 1;
   True SQL: SELECT runner_s__up FROM table_name_67 WHERE margin_of_victory = "5 strokes" AND date = "nov 13, 1988"
   Similarity Score: 0.7410714285714286

   Predicted SQL: SELECT nationality FROM table_name_87 WHERE name = "ivan ivanov" ORDER BY name LIMIT 1;
   True SQL: SELECT nationality FROM table_name_87 WHERE name = "roslinda samsu"
   Similarity Score: 0.7402597402597403

   Predicted SQL: SELECT frequency FROM table_name_28 WHERE multi_1 = "create" ORDER BY frequency LIMIT 1;
   True SQL: SELECT frequency FROM table_name_28 WHERE multi_1 = "7.5×"
   Similarity Score: 0.7397260273972602

   Predicted SQL: SELECT "date" FROM table_203_478 WHERE "week" = 1 ORDER BY "attendance" LIMIT 1 OFFSET 0;
   True SQL: SELECT "opponent" FROM table_203_478 ORDER BY "attendance" DESC LIMIT 1
   Similarity Score: 0.7375

   Predicted SQL: SELECT affiliation FROM table_name_51 WHERE location = "maryland" ORDER BY affiliation LIMIT 1;
   True SQL: SELECT affiliation FROM table_name_51 WHERE location = "issaquah"
   Similarity Score: 0.7375

   Predicted SQL: SELECT result FROM table_name_81 WHERE score = "create" ORDER BY score LIMIT 1;
   True SQL: SELECT result FROM table_name_81 WHERE score = "70-73"
   Similarity Score: 0.7368421052631579

   Predicted SQL: SELECT * FROM table_204_30 WHERE "name" = 'Bowen River Bridge' ORDER BY "year built" LIMIT 1
   True SQL: SELECT "name" FROM table_204_30 WHERE "current location" = 'bressingham steam museum' ORDER BY "year built" DESC LIMIT 1
   Similarity Score: 0.7358490566037735

   Predicted SQL: SELECT SUM(attendance) FROM table_name_94 WHERE result = "create" ORDER BY attendance LIMIT 1
   True SQL: SELECT MAX(attendance) FROM table_name_94 WHERE result = "5-1"
   Similarity Score: 0.7354838709677419

   Predicted SQL: SELECT nfl_team FROM table_name_30 WHERE draft_year = 1985 GROUP BY nfl_team HAVING COUNT(*) = 1;
   True SQL: SELECT nfl_team FROM table_name_30 WHERE draft_year = 1978
   Similarity Score: 0.7354838709677419

   Predicted SQL: SELECT non__stop FROM table_name_92 WHERE plain_stop = "yes" ORDER BY non__stop LIMIT 1 OFFSET 0;
   True SQL: SELECT non__stop FROM table_name_92 WHERE plain_stop = "ㄷ"
   Similarity Score: 0.7354838709677419

   Predicted SQL: SELECT SUM(attendance) FROM table_name_75 WHERE date = "sunday" AND week = 75 GROUP BY date, week;
   True SQL: SELECT MAX(attendance) FROM table_name_75 WHERE date = "august 10, 1963" AND week > 1
   Similarity Score: 0.73224043715847

   Predicted SQL: SELECT grantee FROM table_name_34 WHERE concession = "create" ORDER BY concession LIMIT 1 OFFSET 0;
   True SQL: SELECT grantee FROM table_name_34 WHERE concession = "san ysidro"
   Similarity Score: 0.7317073170731707

   Predicted SQL: SELECT rank FROM table_name_41 WHERE lane = 41 ORDER BY rank LIMIT 1 OFFSET 0;
   True SQL: SELECT rank FROM table_name_41 WHERE lane = 4
   Similarity Score: 0.7317073170731707

   Predicted SQL: SELECT score_1 FROM table_name_46 WHERE tie_no = 46 ORDER BY tie_no LIMIT 1 OFFSET 0;
   True SQL: SELECT score_1 FROM table_name_46 WHERE tie_no = "6"
   Similarity Score: 0.7299270072992701

   Predicted SQL: SELECT make FROM table_name_58 WHERE pos = "create" ORDER BY make LIMIT 1;
   True SQL: SELECT make FROM table_name_58 WHERE pos = 3
   Similarity Score: 0.7288135593220338

   Predicted SQL: SELECT opponent FROM table_name_35 WHERE loss = "create" ORDER BY opponent LIMIT 1;
   True SQL: SELECT opponent FROM table_name_35 WHERE loss = "leal (1–4)"
   Similarity Score: 0.7272727272727273

   Predicted SQL: SELECT surface FROM table_name_16 WHERE score = "15–3" ORDER BY surface LIMIT 1;
   True SQL: SELECT surface FROM table_name_16 WHERE score = "7-5, 5-7, 6-3"
   Similarity Score: 0.7272727272727273

   Predicted SQL: SELECT SUM(wins) FROM table_name_83 WHERE golden_rivers = "the golden rivers" AND against = "the golden rivers" END
   True SQL: SELECT MAX(wins) FROM table_name_83 WHERE golden_rivers = "quambatook" AND against < 1129
   Similarity Score: 0.7254901960784313

   Predicted SQL: SELECT date_won FROM table_name_5 WHERE location = "toronto" AND champion_s_ = "james blake" ORDER BY date_won LIMIT 1;
   True SQL: SELECT date_won FROM table_name_5 WHERE location = "bayamón, puerto rico" AND champion_s_ = "bonecrusher"
   Similarity Score: 0.7232142857142857

   Predicted SQL: SELECT title FROM table_26493520_1 WHERE written_by = "M.A. Numani" AND directed_by = "A.R. Qureshi" ORDER BY title LIMIT 1;
   True SQL: SELECT COUNT(title) FROM table_26493520_1 WHERE written_by = "Alexander Woo" AND directed_by = "Scott Winant"
   Similarity Score: 0.721030042918455

   Predicted SQL: SELECT expected_completion FROM table_name_15 WHERE capacity = "15,000" ORDER BY expected_completion LIMIT 1 OFFSET 0;
   True SQL: SELECT expected_completion FROM table_name_15 WHERE capacity = "22,500"
   Similarity Score: 0.7195767195767195

   Predicted SQL: SELECT aggregate FROM table_name_58 WHERE club = "crete" ORDER BY club LIMIT 1 OFFSET 48;
   True SQL: SELECT aggregate FROM table_name_58 WHERE club = "crvena zvezda"
   Similarity Score: 0.7189542483660131

   Predicted SQL: SELECT home_team FROM table_16388478_4 WHERE home_team = "Melbourne" ORDER BY home_team LIMIT 1;
   True SQL: SELECT home_team AS score FROM table_16388478_4 WHERE home_team = "Fremantle"
   Similarity Score: 0.7167630057803468

   Predicted SQL: SELECT score FROM table_name_60 WHERE partner = "andy roddick" ORDER BY score LIMIT 1;
   True SQL: SELECT score FROM table_name_60 WHERE partner = "albert montañés"
   Similarity Score: 0.7152317880794702

   Predicted SQL: SELECT crew_chief FROM table_1266602_1 WHERE team = "B" ORDER BY crew_chief LIMIT 1;
   True SQL: SELECT COUNT(crew_chief) FROM table_1266602_1 WHERE team = "FAS Lane Racing"
   Similarity Score: 0.7125

   Predicted SQL: SELECT game FROM table_name_76 WHERE road_team = "knicks" AND date = "november 25" ORDER BY game LIMIT 100 OFFSET 0;
   True SQL: SELECT game FROM table_name_76 WHERE road_team = "seattle" AND date = "may 24"
   Similarity Score: 0.711340206185567

   Predicted SQL: SELECT directed_by FROM table_name_55 WHERE written_by = "k.s. rao" ORDER BY directed_by LIMIT 1;
   True SQL: SELECT directed_by FROM table_name_55 WHERE written_by = "robin schwartz & robert tarlow"
   Similarity Score: 0.7096774193548387

   Predicted SQL: SELECT fourth_quarter FROM table_name_86 WHERE rank = "6th" AND second_quarter = "10th" ORDER BY second_quarter LIMIT 1 OFFSET 0;
   True SQL: SELECT fourth_quarter FROM table_name_86 WHERE rank > 3 AND second_quarter = "johnson & johnson 156,515.9"
   Similarity Score: 0.7063829787234043

   Predicted SQL: SELECT status FROM table_name_34 WHERE date = "created" ORDER BY date LIMIT 1;
   True SQL: SELECT status FROM table_name_34 WHERE date = "19/05/1981"
   Similarity Score: 0.7058823529411765

   Predicted SQL: SELECT date_of_birth FROM table_name_81 WHERE name = "maryam dastmalchian" ORDER BY date_of_birth LIMIT 1
   True SQL: SELECT date_of_birth FROM table_name_81 WHERE name = "pananmal punjabi"
   Similarity Score: 0.7045454545454546

   Predicted SQL: SELECT "Title" FROM table_73966 WHERE "Production code" = '8G822' ORDER BY "No." LIMIT 1 OFFSET 0
   True SQL: SELECT "Written by" FROM table_73966 WHERE "Production code" = '2T5954'
   Similarity Score: 0.7023809523809523

   Predicted SQL: SELECT written_by FROM table_22570439_1 WHERE directed_by = "Jesper T. Kyd" ORDER BY written_by LIMIT 1;
   True SQL: SELECT COUNT(written_by) FROM table_22570439_1 WHERE directed_by = "Wendey Stanzler"
   Similarity Score: 0.7021276595744681

   Predicted SQL: SELECT MIN(gold) FROM table_name_54 WHERE nation = "italy" AND bronze = 1 GROUP BY gold, bronze ORDER BY gold LIMIT 1;
   True SQL: SELECT MIN(gold) FROM table_name_54 WHERE nation = "tanzania" AND bronze < 0
   Similarity Score: 0.7010309278350515

   Predicted SQL: SELECT MIN(draws) FROM table_name_38 WHERE goals_against > 0 AND losses = 0 ORDER BY goals_against LIMIT 1 OFFSET 0 + 0 = 1
   True SQL: SELECT SUM(draws) FROM table_name_38 WHERE goals_against = 52 AND losses < 15
   Similarity Score: 0.7

   Predicted SQL: SELECT school FROM table_11677691_11 WHERE player = "Marcus Cox" ORDER BY school LIMIT 1;
   True SQL: SELECT COUNT(school) FROM table_11677691_11 WHERE player = "Derrick Green"
   Similarity Score: 0.6993865030674846

   Predicted SQL: SELECT opponent FROM table_name_98 WHERE result = "create" ORDER BY opponent LIMIT 1 OFFSET 0;
   True SQL: SELECT opponent FROM table_name_98 WHERE result = "0-3"
   Similarity Score: 0.697986577181208

   Predicted SQL: SELECT country FROM table_name_38 WHERE director = "sidney lumet" AND country = "united states" LIMIT 1;
   True SQL: SELECT country FROM table_name_38 WHERE director = "julian schnabel"
   Similarity Score: 0.6976744186046512

   Predicted SQL: SELECT record FROM table_name_38 WHERE score = "1-0" UNIQUE KEY(record) WHERE score = "1-0";
   True SQL: SELECT record FROM table_name_38 WHERE score = "0–2"
   Similarity Score: 0.6944444444444444

   Predicted SQL: SELECT MIN(date) FROM table_name_13 WHERE u_boats_destroyed__kuk_ = "11,636" AND tonnage = 120000000000000000000000000000000000000000000000000000000
   True SQL: SELECT AVG(date) FROM table_name_13 WHERE u_boats_destroyed__kuk_ = "2" AND tonnage = "1,514,050"
   Similarity Score: 0.6938775510204082

   Predicted SQL: SELECT ceremony FROM table_name_67 WHERE category = "best actor in a leading role" ORDER BY ceremony LIMIT 1;
   True SQL: SELECT ceremony FROM table_name_67 WHERE category = "best lyricist"
   Similarity Score: 0.6931818181818182

   Predicted SQL: SELECT author FROM table_name_49 WHERE title = "the little prince" LIMIT 1;
   True SQL: SELECT author FROM table_name_49 WHERE title = "bludgeoning angel dokuro-chan"
   Similarity Score: 0.6928104575163399

   Predicted SQL: SELECT result FROM table_name_26 WHERE party = "conservative" AND first_elected > 1964 ORDER BY first_elected LIMIT 1 OFFSET 0;
   True SQL: SELECT result FROM table_name_26 WHERE party = "republican" AND first_elected < 1856
   Similarity Score: 0.6919431279620853
'''

pattern = re.compile(r"Predicted SQL: (.*?)\n\s*True SQL: (.*?)\n\s*Similarity Score: (.*?)\n", re.DOTALL)

results = []

for match in pattern.finditer(data):
    predicted_query = match.group(1).strip()
    true_query = match.group(2).strip()
    similarity_score = float(match.group(3).strip())

    results.append({
        "predicted_query": predicted_query,
        "true_query": true_query,
        "similarity_score": similarity_score
    })

with open("results.json", "w") as file:
    json.dump(results, file, indent=4)

print("JSON file created successfully!")

JSON file created successfully!


In [20]:
# JSON file ko read karo
with open('results.json', 'r') as json_file:
    data = json.load(json_file)
    print(data)

# # Accuracy values ko collect karo
# accuracies = [entry['accuracy'] for entry in data]

# # Average calculate karo
# average_accuracy = sum(accuracies) / len(accuracies)

# print(f"Average Accuracy: {average_accuracy:.4f}")


[{'predicted_query': 'SELECT rate_limit__p_ FROM table_25316812_1 WHERE desired_rate_change___percentage_ = "10"', 'true_query': 'SELECT rate_limit__p_ FROM table_25316812_1 WHERE desired_rate_change___percentage_ = "+40.4"', 'similarity_score': 0.9726775956284153}, {'predicted_query': 'SELECT position FROM table_26996293_2 WHERE cfl_team = "Edmonton"', 'true_query': 'SELECT position FROM table_26996293_2 WHERE cfl_team = "Edmonton (2)"', 'similarity_score': 0.9701492537313433}, {'predicted_query': 'SELECT name FROM table_name_32 WHERE year = 2010;', 'true_query': 'SELECT name FROM table_name_32 WHERE year = "2001"', 'similarity_score': 0.9494949494949495}, {'predicted_query': 'SELECT position FROM table_name_7 WHERE pick = 7;', 'true_query': 'SELECT position FROM table_name_7 WHERE pick = "146"', 'similarity_score': 0.9306930693069307}, {'predicted_query': 'SELECT voltage_center__v_ FROM table_name_76 WHERE s_spec_number = "110/220v"', 'true_query': 'SELECT voltage_center__v_ FROM tab

In [21]:
# JSON file ko read karo
with open('results.json', 'r') as json_file:
    data = json.load(json_file)

# Similarity score ko collect karo
accuracies = [entry['similarity_score'] for entry in data]

# Average calculate karo
average_accuracy = sum(accuracies) / len(accuracies)

print(f"Average Accuracy: {average_accuracy:.4f}")


Average Accuracy: 0.7702


**FineTunning Model**

In [22]:
with open("results.json", "r") as f:
    data = json.load(f)

fine_tune_data = []
for entry in data:
    fine_tune_data.append({
        "input_text": entry["true_query"],  # ya tum input queries use kar sakte ho agar wo ho
        "output_text": entry["predicted_query"]
    })

with open("fine_tune_data.jsonl", "w") as f:
    for item in fine_tune_data:
        f.write(json.dumps(item) + "\n")

print("Data saved in fine_tune_data.jsonl")


Data saved in fine_tune_data.jsonl


In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"   # ya "google/flan-t5-base" thoda better hai
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [24]:
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq

# Tumhara jsonl dataset load karna
dataset = load_dataset("json", data_files="fine_tune_data.jsonl", split="train")

# Tokenization function
def preprocess_function(examples):
    inputs = examples["input_text"]
    targets = examples["output_text"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [25]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="results.json", split="train")


Generating train split: 0 examples [00:00, ? examples/s]

In [27]:
split_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")  # ya koi bhi model jo tum use kar rahe ho

def tokenize_function(example):
    model_inputs = tokenizer(
        example["predicted_query"],
        text_target=example["true_query"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    return model_inputs


tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [33]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./model_output",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)


TypeError: Accelerator.__init__() got an unexpected keyword argument 'use_seedable_sampler'